In [1]:
import os
import numpy as np
import ngene as ng
import pylab as plt
import tensorflow as tf

/home/gf/packages/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
nx,ny,n_channel = 10,15,1

def data(n):
    x = []
    for i in range(n):
        st = np.random.uniform(0,3)
        x.append(np.random.normal(0,st,(nx,ny)))
    return np.expand_dims(np.array(x),axis=-1)

def truth(x,noise=0):
    return np.std(x,axis=(1,2,3))

def data_provider(n):
    x = data(n)
    y = truth(x)
    return x,y

x,y = data_provider(13)

print(x.shape,y.shape)

(13, 10, 15, 1) (13,)


In [3]:
def architecture(x_in):

    initzer = tf.contrib.layers.xavier_initializer()
    # initzer = tf.contrib.layers.variance_scaling_initializer()

    x = tf.layers.conv2d(x_in,filters=12,
                        kernel_size=5,
                        padding='SAME',
                        kernel_initializer=initzer,
                        kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.05),
                        bias_initializer=tf.constant_initializer(0.1),
                        name='conv_1')
    x = tf.layers.batch_normalization(x,name='norm_1')
    x = tf.nn.relu(x,name='act_1')
    print(x)

    x = tf.layers.conv2d(x,filters=12,
                        kernel_size=5,
                        padding='SAME',
                        kernel_initializer=initzer,
                        kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.05),
                        bias_initializer=tf.constant_initializer(0.1),
                        name='conv_2')
    x = tf.layers.batch_normalization(x,name='norm_2')
    x = tf.nn.relu(x,name='act_2')
    print(x)

    x = tf.layers.conv2d(x,filters=1,
                        kernel_size=5,
                        padding='SAME',
                        kernel_initializer=initzer,
                        kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.05),
                        bias_initializer=tf.constant_initializer(0.1),
                        name='conv_3')
    x = tf.layers.batch_normalization(x,name='norm_3')
    x = tf.nn.relu(x,name='act_3')
    print(x)
    
    x = tf.contrib.layers.flatten(x)
    print(x)
    x = tf.layers.dense(x, 10 , activation=tf.nn.relu)
    print(x)
    y_out = tf.layers.dense(x, 1 ,activation=tf.nn.relu)
    print(y_out)
    
    return y_out

def loss(y_true,x_out):
    return tf.reduce_mean(tf.pow(tf.log(y_true+1) - tf.log(x_out+1), 2))
def loss(y_true,x_out):
    return tf.reduce_mean(tf.pow(y_true - x_out, 2))
model = ng.Model(data_provider,
         restore=False,model_add='./model',arch=architecture, loss = loss)
print('# of variables:',model.n_variables)

Tensor("act_1:0", shape=(?, 10, 15, 12), dtype=float32)
Tensor("act_2:0", shape=(?, 10, 15, 12), dtype=float32)
Tensor("act_3:0", shape=(?, 10, 15, 1), dtype=float32)
Tensor("Flatten/flatten/Reshape:0", shape=(?, 150), dtype=float32)
Tensor("dense/Relu:0", shape=(?, 10), dtype=float32)
Tensor("dense_1/Relu:0", shape=(?, 1), dtype=float32)
# of variables: 5796


In [4]:
model.train(data_provider=data_provider,training_epochs = 10,iterations=200 ,n_s = 127,
                    learning_rate = 0.01, time_limit=None,
                    metric=None, verbose=1,death_preliminary_check = 30,
                    death_frequency_check = 1000)

Epoch:0, cost= 0.200078
Epoch:1, cost= 0.146362
Epoch:2, cost= 0.146524
Epoch:3, cost= 0.145938
Epoch:4, cost= 0.145314
Epoch:5, cost= 0.145632
Epoch:6, cost= 0.147412
Epoch:7, cost= 0.148026
Epoch:8, cost= 0.145958
Epoch:9, cost= 0.146178


In [7]:
x,y = data_provider(10)
y_pred = model.predict(x)

In [8]:
y

array([0.99523418, 0.96065788, 1.00919752, 1.03694964, 1.02378892,
       0.97705924, 1.06203717, 1.07212357, 0.94614618, 0.94411107])

In [9]:
y_pred

array([[0.9855635],
       [0.9855635],
       [0.9855635],
       [0.9855635],
       [0.9855635],
       [0.9855635],
       [0.9855635],
       [0.9855635],
       [0.9855635],
       [0.9855635]], dtype=float32)